# [roberta-base-offensive](https://huggingface.co/cardiffnlp/roberta-base-offensive)

Fine tunning de roberta-base-offensive

In [1]:
from datasets import load_dataset
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

dataset = load_dataset('csv', data_files='./data/train.csv')

In [2]:
dataset['train']

Dataset({
    features: ['label', 'text', 'id'],
    num_rows: 8148
})

In [3]:
dataset = dataset['train'].train_test_split(test_size=0.30)

In [4]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch

# Set the device to CUDA if available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MODEL = f"cardiffnlp/roberta-base-offensive"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL).to(device)
tokenizer.save_pretrained(MODEL)
model.save_pretrained(MODEL)

In [5]:
def tokenize_function(ds):
    return tokenizer(ds["text"], padding="max_length", truncation=True)

In [6]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5703 [00:00<?, ? examples/s]

Map:   0%|          | 0/2445 [00:00<?, ? examples/s]

In [7]:
""" small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000)) """

" small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))\nsmall_eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000)) "

In [8]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="g1-models/roberta-base-offensive-fine", evaluation_strategy="epoch", auto_find_batch_size=True)

In [9]:
import evaluate

metric = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics
)

/home/pablo/anaconda3/envs/final_challenge/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [12]:
trainer.train()

  0%|          | 0/2139 [00:00<?, ?it/s]

  0%|          | 0/4278 [00:00<?, ?it/s]

{'loss': 0.681, 'learning_rate': 4.415614773258532e-05, 'epoch': 0.35}
{'loss': 0.6894, 'learning_rate': 3.831229546517064e-05, 'epoch': 0.7}


  0%|          | 0/306 [00:00<?, ?it/s]

{'eval_loss': 0.6576806902885437, 'eval_accuracy': 0.6372188139059305, 'eval_runtime': 144.1504, 'eval_samples_per_second': 16.961, 'eval_steps_per_second': 2.123, 'epoch': 1.0}
{'loss': 0.6824, 'learning_rate': 3.246844319775596e-05, 'epoch': 1.05}
{'loss': 0.6767, 'learning_rate': 2.6624590930341282e-05, 'epoch': 1.4}
{'loss': 0.6763, 'learning_rate': 2.0780738662926602e-05, 'epoch': 1.75}


  0%|          | 0/306 [00:00<?, ?it/s]

{'eval_loss': 0.6552104949951172, 'eval_accuracy': 0.6372188139059305, 'eval_runtime': 150.9019, 'eval_samples_per_second': 16.203, 'eval_steps_per_second': 2.028, 'epoch': 2.0}
{'loss': 0.6732, 'learning_rate': 1.4936886395511924e-05, 'epoch': 2.1}
{'loss': 0.6692, 'learning_rate': 9.093034128097242e-06, 'epoch': 2.45}
{'loss': 0.6689, 'learning_rate': 3.2491818606825616e-06, 'epoch': 2.81}


  0%|          | 0/306 [00:00<?, ?it/s]

{'eval_loss': 0.6622501611709595, 'eval_accuracy': 0.6372188139059305, 'eval_runtime': 141.5879, 'eval_samples_per_second': 17.268, 'eval_steps_per_second': 2.161, 'epoch': 3.0}
{'train_runtime': 3079.1088, 'train_samples_per_second': 5.556, 'train_steps_per_second': 1.389, 'train_loss': 0.6751329119710401, 'epoch': 3.0}


TrainOutput(global_step=4278, training_loss=0.6751329119710401, metrics={'train_runtime': 3079.1088, 'train_samples_per_second': 5.556, 'train_steps_per_second': 1.389, 'train_loss': 0.6751329119710401, 'epoch': 3.0})